# DEMO of response format capabilities
As you can see in the below made up example, the LLM is FORCED to produce outputs in the format laid out in the class called Test. It's important that Test inherits from Pydantic's BaseModel, as this is handled internally by the openai SDK and is also used to parse the text json output back into the class. Once you have the output object, you can access the attributes of the object like normal. The use of Typing is enforced by openai, ensuring that a string is always a string, an integer is always an integer, a List is always a list, and a Literal has to be one of the options you give it. This is perfect for ensuring LLM outputs are bulletproof in their conforming to your expected outputs, meaning that they do not break unexpectedly no matter what the input is.

The use of Field, from pydantic allows you to add a description property to the attribute. This description is provided to the LLM too, and so are any docstrings!

In [1]:
from pfd_toolkit.llm import LLM
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field, create_model
from typing import List, Literal
load_dotenv()
class Test(BaseModel):
    """An example response format case, this text is seen by the LLM too!"""
    name: Literal['john', 'sam', 'other'] = Field(..., description='The name of the patient')
    sibling_ages: List[str] = Field(..., description='The ages of their siblings')
    death_reason: str = Field(..., description='A verbose reason for the patients death')
    category: Literal['suicide', 'old-age', 'covid'] = Field(..., description='Which category the death reason falls into')

In [2]:
llm = LLM(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-4o-mini')

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
res = llm.generate(prompt='johns siblings are 28 and 29 respectively. john died from confusion and suffered from long-covid which caused cognitive impairment that ultimately resulted in his passing...', response_format=Test)

In [ ]:
res

# Classification of PFD report demo

In [ ]:
class ReportClassification(BaseModel):
    """The classification of the prevention of future death report based on the given context"""
    classification: Literal['suicide', 'depression', 'covid', 'old age', 'confused']

In [ ]:
case = llm.generate(prompt='The cause of death in this case was old age, as the patient lived to an impressive 98 years of age.', response_format=ReportClassification)

In [ ]:
case

In [ ]:
case.classification

My testing of the grammar (response format) results. Please delete or let me know once you are happy and we'll clean the notebook up. Sorry it's here, I started here and don't wanna move it and re do the LLM calls just to regenerate the cell outputs.

In [ ]:
from pfd_toolkit import PFDScraper, llm
from dotenv import load_dotenv
import os
# Load OpenAI API key
load_dotenv("api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_client = llm.LLM(api_key=openai_api_key)

# Run the scraper! :D
scraper = PFDScraper(
    llm=llm_client,
    category="all",
    date_from="2024-01-10",
    date_to="2024-01-11",
    html_scraping=False,
    pdf_fallback=False,
    llm_fallback=True,
    # docx_conversion="LibreOffice", # Doesn't currently seem to work; need to debug.
    include_time_stamp=False,
    delay_range=None,
    verbose=False,
)
scraper.scrape_reports()
#scraper.estimate_api_costs()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
scraper.reports

In [ ]:
sample_report = scraper.reports.iloc[0:1,:]

In [ ]:
print(sample_report.InvestigationAndInquest[0])

In [ ]:
scraper.verbose = True
result = scraper.run_llm_fallback(reports_df=sample_report)

In [ ]:
result